# Basic PageRank
-> work on graphs

In this notebook we will see a basic implementation of the PageRank algorithm, along with some considerations on the complexity. The graphs has been taken from the SNAP (Stanford Large Network Dataset) collection:

https://snap.stanford.edu/data/ : dataset with several types of networks, both directed and undirected + libraries of tools (eg. SNAP has implement algorithms to analyze graphs, even the page rank -but we will implement it from zero)

We will start with a small directed graph called "email-Eu-core network" (https://snap.stanford.edu/data/email-Eu-core.html). It represents the emails sent from user  *i*  to user  *j* (during an observation period). In case of emails with multiple recipients, there is an edge for each recipient. The graph contains 1005 nodes and 25571 edges. There are communities: in the graph we can observe that some nodes are more connnected than others -> identify communities where node have an average number of links higher than outside.

The format is very simple: each line contains an edge **from** one node **to** another node (since it's directed I have a starting node and an ending one)
```text
0 1
2 3
2 4
5 6
5 7
8 9
10 11
```
-> every line is an edge. The list has the 1005 identifier for the nodes, but idk if they are in a sequence (from 0 to 1004)! So I keep a dictionary of them and map the identifier used in the file om the identifier used internally for the nodes and edges -> if the identifier are sequential, in the matrix they can be an index!

From the list, I want to get a square matrix with 1 on column i and row j if ther is a link between node i and j + i has a number of outgoing links equal to d_i.

We first load the data as a list of lists (each line becomes a list with 2 elements), then we will transform the data into a matrix and work with the matrix formulation. After that, we  will transform the data into adjacency list, to improve the efficiency and we reformulate the PageRank accordingly. 

Test an algorithm: find a smaller dataset (=snapshot) to test it and then move to a bigger one.

The diamater of a graph tend to be low, even in big dataset!

## Loading the data


As usual, we first define the function to load the data, adapting such a function to the specific file input format.

In particular, we are going to assign to nodes progressive numbers, so we do not need to rely on the numbering in the file itself (and the node id can be used as matrix index).

In [2]:
def load_data(filename):
    input_lines = []
    raw_lines = open(filename, 'r').read().splitlines() # Read all lines and put them in a list
    num_nodes = 0 # Counter
    nodes = {} # Dictionary of identifier of list and the one I assigned
    for line in raw_lines:
        line_content = line.split() # Each line has two values, so I split them
        from_id = int(line_content[0]) # First node (as int)
        to_id = int(line_content[1]) # Second node (as int)
        if from_id not in nodes:
            nodes[from_id] = num_nodes # Map identifier with internal sequential identifier
            num_nodes += 1
        if to_id not in nodes:
            nodes[to_id] = num_nodes
            num_nodes += 1
        input_lines.append([nodes[from_id], nodes[to_id]]) # Create my list, where each sublist is a pair of nodes with a link
    return input_lines, num_nodes

The input file containing the dataset is called "4-email-Eu-core.txt".

On Colab, remember to mount your Drive
```python
from google.colab import drive
drive.mount('/content/drive')
input_file = "/content/drive/My Drive/..."
```
Let's load our dataset and see its initial content:

In [3]:
input_file = "./4-email-Eu-core.txt"

input_edges, num_nodes = load_data(input_file) # Load data

print("\nThe dataset contains", num_nodes, "nodes and", # The result is the list of edges (=sublist) + number of nodes
      len(input_edges),"edges.\n")
print("The first five edges are:", input_edges[:5],"\n")


The dataset contains 1005 nodes and 25571 edges.

The first five edges are: [[0, 1], [2, 3], [2, 4], [5, 6], [5, 7]] 



## Matrix formulation

Building and maintaining in memory the full adjacency matrix is inefficient, since the matrix is sparse. But working with a matrix is more intuitive, therefore we will start with this approach. 

As done before, we will use the Numpy library for handling matrixes. We first fill the matrix with "1" if there is an edge, then we will transform the matrix into a column stochastic one.

Once I have the matrix formulation. I can use the power iteration method:
r^(i+1) = M r^(i)

The matrix has to be column stochastics in order to not have leaks! So I fill the columns with 1/N (where N is the number of nodes)

?????? min 24 fin 26.15

In [4]:
import numpy as np

# create an NxN matrix of zeros, where N = number of nodes 
matrix_nodes = np.zeros((num_nodes, num_nodes)) # That's why I returned the number of nodes above

# Set element i,j to "1" if there is an edge from j to i
for edge in input_edges: # For each elemnt of the list I read the from i and to j -> put a 1 in row j and column i
    from_id = edge[0]
    to_id = edge[1]
    matrix_nodes[to_id, from_id] = 1 # Matrix filled with 1
    
# compute the "sparsity", i.e., percentage of non-zero cells
sparsity = 100*float(np.count_nonzero(matrix_nodes))/float(num_nodes*num_nodes) # Computed for statistics
print("Sparsity: %.2f%%" % (sparsity))

# Show a snippet of the matrix
matrix_nodes

Sparsity: 2.53%


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

I could have guessed the sparsity knowing the matrix has dimension 10^6 + 25k edges and 1K nodes -> 25k/10^6= 25.3%

We normalize each element with the sum of the column. If the column has no entry (i.e., no outgoing links, it's a dead-end), we fill each element with 1/N.

In [6]:
# Normalize the matrix: first I check if there is at least a value different from zero in each column
for col in range(num_nodes):
    degree = np.sum(matrix_nodes[:,col])  
    if degree > 0: # For each column, take the sum of its values: if it's >0 there is at least a 1, I have to normalize -> 1/d_i
        matrix_nodes[:,col] *= 1.0/degree
    else:
        matrix_nodes[:,col] = 1.0/num_nodes # In this case, I put 1/N

# Show a snippet of the normalized matrix
matrix_nodes

array([[2.43902439e-02, 0.00000000e+00, 0.00000000e+00, ...,
        9.95024876e-04, 0.00000000e+00, 9.95024876e-04],
       [2.43902439e-02, 1.00000000e+00, 0.00000000e+00, ...,
        9.95024876e-04, 0.00000000e+00, 9.95024876e-04],
       [0.00000000e+00, 0.00000000e+00, 1.19047619e-02, ...,
        9.95024876e-04, 0.00000000e+00, 9.95024876e-04],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.95024876e-04, 0.00000000e+00, 9.95024876e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.95024876e-04, 0.00000000e+00, 9.95024876e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.95024876e-04, 0.00000000e+00, 9.95024876e-04]])

## Preliminary questions

PRIMI 10 MIN 15/05

### Question  Q1
<div class="alert alert-info">
Consider the matrix before the normalization: compute the number of dead-end nodes (and its percentage with respect to the total number of nodes).
</div>

In [7]:
#  Dead-end nodes = column sum up to 0 = column has no 1

### Question  Q2
<div class="alert alert-info">
Compute the sparsity of the matrix (before and after normalization), i.e., the ratio between the non-zero elements and the total number of elements.
</div>

In [ ]:
# your answer

### Question  Q3
<div class="alert alert-info">
Compute the amount of memory (bytes) required to store the matrix, before and after the normalization (then I will check with the adjency list too -> for larger dataset the memory is not enough to store the matrix, so I have to use the list)
</div>

In [ ]:
# your answer

### Question  Q4
<div class="alert alert-info">
Compute some basic statistics about the graph, such as:
    
- minimum, maximum and average outdegree (number of outgoing links);
- outdegree distribution (percentage of nodes with outdegree 0, 1, 2, 3, ...): graph with number of nodes that have 0 links, then 1, then etc (histogram)
- minimum, maximum and average indegree (number of incoming links);
- indegree distribution (percentage of nodes with indegree 0, 1, 2, 3, ...): can be different from the outdegree distribution
</div>

In [ ]:
# Get a feeling of the statistic of the dataset 
# adg_matrix is the matrix with NO normalization

# nz_indexes is a data structure that contains the indexes of non-zero values
# [-1] is the last element

In [ ]:
# Don't plot up to the last value, but  focus for the first 100


Corresponding to 0 are the dead-end nodes (highest number of nodes), with an high putdegree I get less and less nodes


In [1]:
# Same for the indegree, summing over columns and not rows now

I expect the same average, but min and max different

In [ ]:
# Plot as above

Now the 1 has the most nodes, not 0.

# Pagerank (matrix formulation)

We are now ready to compute the Pagerank with the power iteration approach. The inputs are:

- The adjacency matrix;
- The teleport parameter beta;
- The target error;
- The maximum number of iterations.


In [14]:
# Implement the powe iteration method (with beta): as input I have the matrix itself -> there are no dead end since I replaced empty
# values with 1/N; but I may have spder traps, so I use teleport
def matrix_pagerank(input_matrix, beta, target_error, max_iterations): # max_iterations is a stop condition: from the theory I hould stop when the ranking of i+1 is equal to the rank of i
    #                                                                    -> I have reached a stable situation. But there I have numerical approximation, so this never hapens! I have to limit
    #                                                                    the number of iterations.
    #                                                                    I also have target_error to reach a stable solution: whatever happen first, I stop :)
    # Since the matrix is an input, I can infer the number of nodes from the matrix size
    num_nodes = input_matrix.shape[0]
    # Initialize the ranking vector, it's r^(i)
    # Shape of vector = number of nodes
    rank_prev = np.full((num_nodes), 1.0/num_nodes) # Create initial ranking with 1/N assigned to all nodes
    # Iterate at most "max_iterations" times
    for curr_iteration in range(max_iterations):
        # It's r^(i+1)
        rank_new = beta*input_matrix.dot(rank_prev) + (1.0-beta)/num_nodes # All operation on vectors or matrixes -> I get a new vector at the end
        # Compute the error
        curr_err = np.sum(abs(rank_new - rank_prev)) # Sum up all elements
        # For debugging: print the error at each iteration 
        print("iteration:", curr_iteration, ", err:", curr_err)
        if curr_err < target_error: # If I reach the target error I stop, otherwise I do iterations until max_iterations (but I always stop)
            break
        # Note the ".copy()", otherwise they end up to be the same vector (=override)
        rank_prev = rank_new.copy()
    return rank_new, curr_err, curr_iteration

We run the PageRank with the following parameters:

In [13]:
beta = 0.8
error = 0.0001
max_iterations = 30

pg_nodes, err, iterations = matrix_pagerank(matrix_nodes, beta, error, max_iterations)
print("\nComputed", iterations, "iterations with final error", err, "\n")
print(pg_nodes)

iteration: 0 , err: 0.4982592433502846
iteration: 1 , err: 0.11132642279660003
iteration: 2 , err: 0.02938698563919208
iteration: 3 , err: 0.011919997309565854
iteration: 4 , err: 0.006469027782158059
iteration: 5 , err: 0.0044034387718768705
iteration: 6 , err: 0.003182390530861841
iteration: 7 , err: 0.002358449708309306
iteration: 8 , err: 0.001801200071351313
iteration: 9 , err: 0.0013987204734422808
iteration: 10 , err: 0.001095372908458477
iteration: 11 , err: 0.0008663391375282711
iteration: 12 , err: 0.0006876723577637379
iteration: 13 , err: 0.0005459417455002378
iteration: 14 , err: 0.0004334705824646937
iteration: 15 , err: 0.00034419929815731904
iteration: 16 , err: 0.00027333072070186333
iteration: 17 , err: 0.00021706422712188943
iteration: 18 , err: 0.00017238690049600547
iteration: 19 , err: 0.0001369091801939733
iteration: 20 , err: 0.00010873521913732417
iteration: 21 , err: 8.636047538247281e-05

Computed 21 iterations with final error 8.636047538247281e-05 

[0.0012

As I do more iterations, the error decrease.

???? min 50 fin min 54

Inspecting the rank help understand if the rank was set corectly.

### Question  Q5
<div class="alert alert-info">
Modify the function so that the error is not an absolute value (such as 0.0001), but a relative one for each node rank. For instance, one could stop the iterations if, for each node, the variation of the rank is below 1% with respect to the previous rank.
</div>

In [15]:
# The error goes from absolute value to percentage: at each iteration for each row (=node) the error must be lower than 1%.
# If all rankings of the nodes are <1%, indipendently from the rank, I can stop


## Adjacency list formulation

This formulation requires the whole matrix! If no memory -> adjacency list. Instead of a matrix, we maintain a data structure (dicionary) in which, for each node (key), we have a list of neighbors (value).

In [16]:
adj_nodes = {} # list of neighbors as a dictionary
for edge in input_edges: # Go through the list of edges I computed at the start
    from_id = edge[0] # From node
    to_id = edge[1]
    # ???? MIN 57 fin 5
    if from_id not in adj_nodes:
        adj_nodes[from_id] = [to_id]
    else:
        adj_nodes[from_id].append(to_id)

# For simplicity, we sort each value (not necessary, but better for visualization)
for node in adj_nodes:
    adj_nodes[node].sort()

# Show the neighbors of the first node
print(adj_nodes[0])

[0, 1, 5, 6, 17, 18, 64, 73, 74, 88, 101, 103, 146, 148, 166, 177, 178, 215, 218, 221, 222, 223, 226, 238, 248, 250, 266, 268, 283, 297, 309, 313, 316, 368, 377, 380, 459, 498, 560, 581, 734]


We can now change our Pagerank formulation considering as input the adjacency list. Note that, differently from the matrix, we need to explicitly indicate the number of nodes, beacuse nodes with no outgoing links (dead-ends) do not appear (as key) in the adjacency list and we cannot not infer from the size of the list the number of nodes.

Note also that we evaluate at each step how much ranking has been lost (due to dead-ends), and we reintegrate it.

In [18]:
# Given a node, for the PR we consider the incoming links. But the adj list has the outgoing list, so I build the rank incrementally.
# Also, a node with no outgoing links is not gonna appear in my adj list! Then the lost vector will be in the vector and have a rank,
# but that ranking will be not distributed and will be lost. To solve this, I start from a ranking equal to 1 and subtracts the previous
# ranks computed -> at the end I will know which rank was lost ????? MIN 1.05
def adj_pagerank(adj_list, num_nodes, beta, target_error, max_iterations): # num_nodes is to build an empty vector of that size
    # Initialize the ranking vector with 1/N
    rank_prev = np.full((num_nodes), 1.0/num_nodes)
    # Iterate at most "max_iterations" times
    for curr_iteration in range(max_iterations):
        # since rank_new is incrementally built every time, it has to be initialized
        rank_new = np.zeros((num_nodes)) # start from 0 and add incrementally all the values, going element by element
        # The leaked ranking is found decrementally
        leaked = 1.0 # start from here and for each node we substract from it: the fraction remaining at the end is the node lost
        for node in adj_list:
            # We derive the outdegree from the list size
            outdegree = len(adj_list[node])
            leaked -= rank_prev[node]
            for neigh in adj_list[node]:
                rank_new[neigh] += beta*rank_prev[node]/outdegree # Update incrementally
        # Add the teleport (1-beta) and the leaked values (times beta)
        rank_new += (1.0-beta+beta*leaked)/num_nodes
        # Compute the error
        curr_err = np.sum(abs(rank_new - rank_prev))
        if curr_err < target_error:
            break
        rank_prev = rank_new.copy()
    return rank_new, curr_err, curr_iteration

Let's try it and compare with the results obtained with the matrix formulation:


In [20]:
pg_nodes_adj, err, iterations = adj_pagerank(adj_nodes, num_nodes, beta, error, max_iterations)
print("\nComputed", iterations, "iterations with final error", err, "\n")
print(pg_nodes_adj)

print("\nThe sum of the element-wise difference with the previous formulation is:\n", 
      np.sum(abs(pg_nodes_adj - pg_nodes)), "\n") # check that the 2 formulations are similar enough (Not 0 due to precision errors)


Computed 21 iterations with final error 8.636047538305758e-05 

[0.00127608 0.00743391 0.00204666 ... 0.0002516  0.00025427 0.00025765]

The sum of the element-wise difference with the previous formulation is:
 1.2421704290166424e-15 



## Additional questions

### Question  Q6
<div class="alert alert-info">
Compute the amount of memory (bytes) required to store the adjacency list, and compare it with the memory used by the matrix. 
</div>

In [2]:
# Memory used by the matrix is in Q3, get the mem of the adj list in the same way

Raw memory for the adj list is WAY less than the matrix (which is more intuitive though)

### Question  Q7
<div class="alert alert-info">
Find the top 5 ranked nodes, the bottom 5 ranked nodes, and compute the ranking range (difference between the highest and lowest ranking).
</div>

In [3]:
# Get a sense of the values computed

# [i] get the top node, [-(i+1)] the corresponding bottom node

### Question  Q8
<div class="alert alert-info">
Divide the different ranking values into ranges (with constant size, or exponentially increasing size) and show the percentage of nodes in each range. 
    
Compare the distribution with the indegree distribution computed before.
</div>

In [4]:
# Divide the values in range and count how many nodes in each range -> can plot this
# I divide in 100 intervals

In [ ]:
# Plot

### Question  Q9
<div class="alert alert-info">
Analyze a larger graph (file "4-soc-Epinions1.txt"), and in particular:
    
- How many nodes and edges does it have?
- Is your system able to handle the PageRank computation using the matrix formulation?
- Compute the PageRank with the adjancency list formulation.
- Reply to Q7-Q8 for this graph.
</div>

In [ ]:
# your answer here

### Question  Q10
<div class="alert alert-info">
Can you reformulate the PageRank algorithm assuming to have the adjacency list of the incoming links? 
    
Does this formulation has benefits?
</div>

In [ ]:
# your answer here